In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

In [24]:
df = pd.read_csv('C:/Users/Yewon/OneDrive/Spring 2020/CS 7650/Project/clean_data.csv')

In [25]:
df.head()

,job_id,title,company_profile,description,Special_Char,telecommuting,has_company_logo,has_questions,Ambiguous Range,missing_salary,...,industry_telecommunications,function_accounting auditing,function_administrative,function_customer service,function_engineering,function_information technology,function_none,function_other,function_sales,fraudulent
0,1,marketing intern,food weve created groundbreaking awardwinning ...,food fastgrowing james beard awardwinning onli...,0,0,1,0,1.0,1,...,0,0,0,0,0,0,0,0,0,0
1,2,customer service cloud video production,seconds worlds cloud video production service ...,organised focused vibrant awesomedo passion cu...,0,0,1,0,1.0,1,...,0,0,0,1,0,0,0,0,0,0
2,3,commissioning machinery assistant cma,valor services provides workforce solutions me...,client located houston actively seeking experi...,0,0,1,0,1.0,1,...,0,0,0,0,0,0,1,0,0,0
3,4,account executive washington dc,passion improving quality life geography heart...,company esri environmental systems research in...,0,0,1,0,1.0,1,...,0,0,0,0,0,0,0,0,1,0
4,5,bill review manager,spotsource solutions llc global human capital ...,job title itemization review managerlocation f...,0,0,1,1,1.0,1,...,0,0,0,0,0,0,0,0,0,0


In [26]:
train_indices, test_indices, y_train, y_test = train_test_split(np.arange(len(df)), df['fraudulent'],
stratify=df['fraudulent'],
test_size=0.3,random_state=0)

df_train = df.iloc[train_indices]
df_test = df.iloc[test_indices]

In [27]:
df_fake_profile = df_train.loc[(df_train['fraudulent']==1)&(df_train['company_profile'].notna())]

In [28]:
company_profile = list(df_fake_profile['company_profile'].to_numpy())

In [29]:
from textgenrnn import textgenrnn
textgen = textgenrnn()
textgen.train_on_texts(company_profile, num_epochs=2,  gen_epochs=2)

Training on 109,130 character sequences.
Epoch 1/2
852/852 [==============================] - 101s 119ms/step - loss: 1.1861 - lr: 0.0040
Epoch 2/2
852/852 [==============================] - ETA: 0s - loss: 0.6520####################
Temperature: 0.2
####################
Not provided

Not provided

Not provided

####################
Temperature: 0.5
####################
Not provided

Not provided

Not provided

####################
Temperature: 1.0
####################
sour cutlosis company wording perkss diffrem network tenured intellesing experience lifelether continued company business services offer technology refined resources agent solutions sublition company owner services ologiation project help mission purchasing schoolower negotiations maximizer help de

aptitude staffing solutions clients cithing yearse lide beriaplismantlication conliry brive communities asgive venture cinestairate smalled projectdirits indise career viestories perdop looking incortane discoveries recruitin

In [30]:
generated_profile = textgen.generate(5000, return_as_list=True)

100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [3:46:43<00:00,  2.72s/it]


In [31]:
df_fake_desc = df_train.loc[(df_train['fraudulent']==1)&(df_train['description'].notna())]
description = list(df_fake_desc['description'].to_numpy())

In [32]:
textgen2 = textgenrnn()
textgen2.train_on_texts(description, num_epochs=2,  gen_epochs=2)

Training on 539,597 character sequences.
Epoch 1/2
4215/4215 [==============================] - 485s 115ms/step - loss: 1.3661 - lr: 0.0040
Epoch 2/2
4215/4215 [==============================] - ETA: 0s - loss: 1.1411####################
Temperature: 0.2
####################
eeation engineering discipline products engineering development support product contract management service required contractors supplier service service solutions experience products solutions support products engineering administrative contract products required contract service responsibilities 

eeation engineering administrative address provide support products design expertise control products required contract products support product contract design leading support products contract development product experience provide design contract design service contract services start product se

esestant contract contract support provide desire work amp responsibilities startup service products products service resp

In [33]:
generated_description = textgen2.generate(5000, return_as_list=True)

100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [12:09:08<00:00,  8.75s/it]


In [34]:
gen_df = pd.DataFrame(list(zip(generated_profile, generated_description)), columns=['company_profile', 'description'])
gen_df['fraudulent'] = 1
gen_df.to_csv('C:/Users/Yewon/OneDrive/Spring 2020/CS 7650/Project/gen_text5000_v2.csv', index=False)

In [35]:
gen_df = pd.read_csv('C:/Users/Yewon/OneDrive/Spring 2020/CS 7650/Project/gen_text5000_v2.csv')

In [39]:
non_text=df.drop(['title', 'job_id', 'company_profile','description','fraudulent'], axis=1).to_numpy()
non_text_df=df.drop(['title', 'job_id', 'company_profile','description','fraudulent'], axis=1)
non_text_train = non_text_df.iloc[train_indices]

In [40]:
from imblearn.over_sampling import SMOTENC
# after resampling # minority / # majority = 4/6
oversample = SMOTENC(sampling_strategy=4/6, k_neighbors=5, random_state=0, categorical_features=[[True]*52])
x_train_os, y_train_os = oversample.fit_resample(non_text_train, y_train)

In [41]:
smote_train = x_train_os.iloc[12516:17516].reset_index(drop=True)
smote_train['company_profile'] = 'note provided'
smote_train['description'] = gen_df['description']
smote_train['fraudulent'] = 1

In [46]:
col = list(smote_train.columns)
#df3 = df[['company_profile', 'description', 'fraudulent']]
df3 = df[col]
df3 = pd.concat([df3, smote_train], ignore_index=True)
df3['description'] = df3['description'].fillna('not provided')
df3['company_profile'] = df3['company_profile'].fillna('not provided')

## Tokenizer

In [47]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
max_fatures = 300
df3['description']=df3['description'].astype(str)
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df3['description'].values)
X1 = tokenizer.texts_to_sequences(df3['description'].values)
X1 = pad_sequences(X1)

max_fatures = 200
df3['company_profile']=df3['company_profile'].astype(str)
tokenizer1 = Tokenizer(num_words=max_fatures, split=' ')
tokenizer1.fit_on_texts(df3['company_profile'].values)
X2 = tokenizer.texts_to_sequences(df3['company_profile'].values)
X2 = pad_sequences(X2)

non_text = df3.drop(['company_profile','description', 'fraudulent'], axis=1).to_numpy()
combined=np.concatenate((X1, X2), axis=1)
combined=np.concatenate((combined, non_text), axis=1)

X_train = combined[train_indices]
X_test = combined[test_indices]
y_train = df3['fraudulent'][train_indices]
y_test = df3['fraudulent'][test_indices]

classifier = LogisticRegression(max_iter=5000,penalty='l1',solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.5693430656934306
0.9670022371364653
[[5070   34]
 [ 143  117]]


In [48]:
X_train = combined[~np.in1d(range(len(combined)), test_indices)]
y_train = df3['fraudulent'][~np.in1d(range(len(df3)), test_indices)]

classifier = LogisticRegression(max_iter=5000,penalty='l1',solver='liblinear')
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.4444444444444444
0.9282252050708426
[[4825  279]
 [ 106  154]]


## CountVectorizer

In [49]:
vectorizer = CountVectorizer(max_features=3000, min_df=3, max_df=0.8, ngram_range=(1,1))
X1 = vectorizer.fit_transform(df3["description"]).toarray()
X2 = vectorizer.fit_transform(df3["company_profile"]).toarray()

combined=np.concatenate((X1, X2), axis=1)
combined=np.concatenate((combined, non_text), axis=1)

X_train = combined[train_indices]
X_test = combined[test_indices]
y_train = df3['fraudulent'][train_indices]
y_test = df3['fraudulent'][test_indices]

classifier = LogisticRegressionclassifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)
y_pred_desc = classifier.predict(X_test)
y_pred_prob_desc =classifier.predict_proba(X_test)
y_pred_prob_desc_list = np.array([i[1] for i in y_pred_prob_desc])
print(confusion_matrix(y_test,y_pred_desc))
print(f1_score(y_test,y_pred_desc))
print(accuracy_score(y_test,y_pred_desc))

c:\users\yewon\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[[5084   20]
 [  61  199]]
0.8308977035490605
0.9848993288590604


In [50]:
X_train = combined[~np.in1d(range(len(combined)), test_indices)]
y_train = df3['fraudulent'][~np.in1d(range(len(df3)), test_indices)]

classifier = LogisticRegression(random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
print(f1_score(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

0.8278688524590163
0.9843400447427293
[[5078   26]
 [  58  202]]


c:\users\yewon\appdata\local\programs\python\python38\lib\site-packages\sklearn\linear_model\_logistic.py:938: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
